In [1]:
    import hopsworks
    import pandas as pd
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    import seaborn as sns
    from matplotlib import pyplot
    from hsml.schema import Schema
    from hsml.model_schema import ModelSchema
    import joblib
    import os
    
    
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.metrics import mean_squared_error

/Users/deepakshankar/opt/anaconda3/envs/ML_DL/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
# You have to set the environment variable 'HOPSWORKS_API_KEY' for login to succeed
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/210070
Connected. Call `.close()` to terminate connection gracefully.


In [6]:
# The feature view is the input set of features for your model. The features can come from different feature groups.    
# You can select features from different feature groups and join them together to create a feature view
wine_fg = fs.get_feature_group(name="wine", version=1)
query = wine_fg.select_all()
feature_view = fs.get_or_create_feature_view(name="wine",
                                  version=1,
                                  description="Read from Wine quality dataset",
                                  labels=["quality"],
                                  query=query)

In [7]:
#THIS IS MULTI CLASS CLASSIFICATION, AND NOT REGRESSION. AS THE TARGET VARIABLE(QUALITY) IN AN INTERGER BETWEEN 2-8.
# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

# # Train our model with the Scikit-learn K-nearest-neighbors algorithm using our features (X_train) and labels (y_train)
# model = KNeighborsClassifier(n_neighbors=2)
# model.fit(X_train, y_train.values.ravel())


# Initialize the RandomForestRegressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train, y_train.values.ravel())

Finished: Reading data from Hopsworks, using Hive (4.96s) 


RandomForestRegressor(random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier


# You can read training data, randomly split into train/test sets of features (X) and labels (y)        
X_train, X_test, y_train, y_test = feature_view.train_test_split(0.2)

# # Train our model with the Scikit-learn K-nearest-neighbors algorithm using our features (X_train) and labels (y_train)
# model = KNeighborsClassifier(n_neighbors=2)
# model.fit(X_train, y_train.values.ravel())


# Initialize the RandomForestRegressor
rf_model = RandomForestClassifier(n_estimators=200, random_state=43)

# Fit the model to the training data
rf_model.fit(X_train, y_train.values.ravel())

Finished: Reading data from Hopsworks, using Hive (11.94s) 


RandomForestClassifier(n_estimators=200, random_state=43)

In [27]:
import numpy as np

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 0.7754439507956175


In [29]:
X_train

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol
0,1,6.9,0.29,0.32,5.8,0.040,16.0,112.0,0.99300,3.04,0.58,11.2
3,1,6.7,0.22,0.33,1.2,0.036,36.0,86.0,0.99058,3.10,0.76,11.4
4,1,8.3,0.14,0.45,1.5,0.039,18.0,98.0,0.99215,3.02,0.56,11.0
5,1,6.4,0.15,0.36,1.8,0.034,43.0,150.0,0.99220,3.42,0.69,11.0
6,1,7.1,0.12,0.30,3.1,0.018,15.0,37.0,0.99004,3.02,0.52,11.9
...,...,...,...,...,...,...,...,...,...,...,...,...
5324,1,7.6,0.19,0.37,13.1,0.033,52.0,151.0,0.99726,3.18,0.79,10.4
5325,1,6.7,0.24,0.37,11.3,0.043,64.0,173.0,0.99632,3.08,0.53,9.9
5326,1,6.0,0.27,0.19,1.7,0.020,24.0,110.0,0.98980,3.32,0.47,12.6
5327,1,6.8,0.33,0.31,7.4,0.045,34.0,143.0,0.99226,3.06,0.55,12.2


In [30]:
y_train

,quality
0,5
3,6
4,6
5,8
6,7
...,...
5324,6
5325,6
5326,7
5327,6


In [31]:
X_test

,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol
1,1,7.4,0.28,0.25,11.9,0.053,25.0,148.0,0.99760,3.10,0.62,9.2
2,1,7.5,0.24,0.31,13.1,0.050,26.0,180.0,0.99884,3.05,0.53,9.1
12,0,7.8,0.59,0.18,2.3,0.076,17.0,54.0,0.99750,3.43,0.59,10.0
14,1,6.1,0.32,0.24,1.5,0.036,38.0,124.0,0.98980,3.29,0.42,12.4
16,1,7.1,0.36,0.56,1.3,0.046,25.0,102.0,0.99230,3.24,0.33,10.5
...,...,...,...,...,...,...,...,...,...,...,...,...
5298,1,6.3,0.22,0.57,1.4,0.208,14.0,96.0,0.99320,3.16,0.43,10.0
5303,1,7.9,0.25,0.34,11.4,0.040,53.0,202.0,0.99708,3.11,0.57,9.6
5315,1,8.9,0.27,0.28,0.8,0.024,29.0,128.0,0.98984,3.01,0.35,12.4
5316,1,6.6,0.35,0.29,14.4,0.044,54.0,177.0,0.99910,3.17,0.58,8.9


In [33]:
y_pred

array([5, 6, 5, ..., 6, 5, 5])

In [28]:
# Compare predictions (y_pred) with the labels in the test set (y_test)
metrics = classification_report(y_test, y_pred, output_dict=True)

In [35]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# # Assuming X_train, X_test, y_train, y_test are your feature and target variables
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Linear Regression
linear_reg = LinearRegression()
linear_reg.fit(X_train, y_train.values.ravel())
y_pred_linear = linear_reg.predict(X_test)
rmse_linear = np.sqrt(mean_squared_error(y_test, y_pred_linear))
print("Linear Regression RMSE:", rmse_linear)

# Support Vector Regression (SVR)
svr = SVR()
svr.fit(X_train, y_train.values.ravel())
y_pred_svr = svr.predict(X_test)
rmse_svr = np.sqrt(mean_squared_error(y_test, y_pred_svr))
print("SVR RMSE:", rmse_svr)

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train.values.ravel())
y_pred_gbr = gbr.predict(X_test)
rmse_gbr = np.sqrt(mean_squared_error(y_test, y_pred_gbr))
print("Gradient Boosting Regressor RMSE:", rmse_gbr)


Linear Regression RMSE: 0.7443900733559765
SVR RMSE: 0.8225495891515717
Gradient Boosting Regressor RMSE: 0.7257877114546966


In [36]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'wine_model' directory will be saved to the model registry. Create the dir, first.
model_dir="wine_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save our rf_model(random forest) to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(rf_model, model_dir + "/wine_model.pkl")   

# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
wine_model = mr.python.create_model(
    name="wine_model", 
    metrics={"rmse" : rmse},
    model_schema=model_schema,
    description="Wine Quality Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
wine_model.save(model_dir)

Connected. Call `.close()` to terminate connection gracefully.


Uploading model files (0 dirs, 0 files):  17%|▋   | 1/6 [00:00<00:02,  2.03it/s]
Uploading: 0.000%|                        | 0/49250481 elapsed<00:00 remaining<?
Uploading: 2.129%|▎             | 1048576/49250481 elapsed<00:03 remaining<02:28
Uploading: 4.258%|▌             | 2097152/49250481 elapsed<00:03 remaining<01:04
Uploading: 6.387%|▉             | 3145728/49250481 elapsed<00:08 remaining<02:21
Uploading: 8.516%|█▏            | 4194304/49250481 elapsed<00:10 remaining<01:48
Uploading: 12.774%|█▋           | 6291456/49250481 elapsed<00:10 remaining<00:54
Uploading: 14.903%|█▉           | 7340032/49250481 elapsed<00:12 remaining<00:54
Uploading: 19.162%|██▍          | 9437184/49250481 elapsed<00:12 remaining<00:31
Uploading: 21.291%|██▌         | 10485760/49250481 elapsed<00:13 remaining<00:35
Uploading: 23.420%|██▊         | 11534336/49250481 elapsed<00:14 remaining<00:26
Uploading: 27.678%|███▎        | 13631488/49250481 elapsed<00:15 remaining<00:26
Uploading: 36.194%|████▎    

Model created, explore it at https://c.app.hopsworks.ai:443/p/210070/models/wine_model/1


Model(name: 'wine_model', version: 1)